In [124]:
import pandas as pd
import numpy as np

# FIFA Prediction Analysis

In [125]:
fifa_15 = pd.read_csv("./fifa_players/players_15.csv")
fifa_16 = pd.read_csv("./fifa_players/players_16.csv")
fifa_17 = pd.read_csv("./fifa_players/players_17.csv")
fifa_18 = pd.read_csv("./fifa_players/players_18.csv")
fifa_19 = pd.read_csv("./fifa_players/players_19.csv")
fifa_20 = pd.read_csv("./fifa_players/players_20.csv")
fifa_15.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,27,1987-06-24,169,67,Argentina,FC Barcelona,...,62+3,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,29,1985-02-05,185,80,Portugal,Real Madrid,...,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3
2,9014,https://sofifa.com/player/9014/arjen-robben/15...,A. Robben,Arjen Robben,30,1984-01-23,180,80,Netherlands,FC Bayern München,...,64+3,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3
3,41236,https://sofifa.com/player/41236/zlatan-ibrahim...,Z. Ibrahimović,Zlatan Ibrahimović,32,1981-10-03,195,95,Sweden,Paris Saint-Germain,...,61+3,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3
4,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Neuer,28,1986-03-27,193,92,Germany,FC Bayern München,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
defender = ["LWB", "LB", "CB", "RB", "RWB"]
midfielder = ["CDM", "CAM", "CM", "LM", "RM"]
forward = ["ST", "CF", "LW", "LF", "RW", "RF"]

fifa_15 = fifa_15.drop(["sofifa_id", "nation_position", "player_url", "long_name", "dob", "potential", "value_eur", "wage_eur", "international_reputation", "body_type", "real_face", "release_clause_eur", "player_tags", "loaned_from", "joined", "contract_valid_until", "height_cm", "weak_foot", "skill_moves", "work_rate", "team_jersey_number", "weight_kg", "age", "player_positions", "preferred_foot", "nation_jersey_number"], axis = 1)
fifa_15 = fifa_15.iloc[:, range(0, 17)]
fifa_15 = fifa_15.rename(columns = {"short_name": "name", "team_position": "position", "physic": "physical"})

fifa_16 = fifa_16.drop(["sofifa_id", "player_url", "nation_position", "long_name", "dob", "potential", "value_eur", "wage_eur", "international_reputation", "body_type", "real_face", "release_clause_eur", "player_tags", "loaned_from", "joined", "contract_valid_until", "height_cm", "weak_foot", "skill_moves", "work_rate", "team_jersey_number", "weight_kg", "age", "player_positions", "preferred_foot", "nation_jersey_number"], axis = 1)
fifa_16 = fifa_16.iloc[:, range(0, 17)]
fifa_16 = fifa_16.rename(columns = {"short_name": "name", "team_position": "position", "physic": "physical"})

fifa_17 = fifa_17.drop(["sofifa_id", "player_url", "nation_position", "long_name", "dob", "potential", "value_eur", "wage_eur", "international_reputation", "body_type", "real_face", "release_clause_eur", "player_tags", "loaned_from", "joined", "contract_valid_until", "height_cm", "weak_foot", "skill_moves", "work_rate", "team_jersey_number", "weight_kg", "age", "player_positions", "preferred_foot", "nation_jersey_number"], axis = 1)
fifa_17 = fifa_17.iloc[:, range(0, 17)]
fifa_17 = fifa_17.rename(columns = {"short_name": "name", "team_position": "position", "physic": "physical"})

fifa_18 = fifa_18.drop(["sofifa_id", "player_url", "nation_position", "long_name", "dob", "potential", "value_eur", "wage_eur", "international_reputation", "body_type", "real_face", "release_clause_eur", "player_tags", "loaned_from", "joined", "contract_valid_until", "height_cm", "weak_foot", "skill_moves", "work_rate", "team_jersey_number", "weight_kg", "age", "player_positions", "preferred_foot", "nation_jersey_number"], axis = 1)
fifa_18 = fifa_18.iloc[:, range(0, 17)]
fifa_18 = fifa_18.rename(columns = {"short_name": "name", "team_position": "position", "physic": "physical"})

fifa_19 = fifa_19.drop(["sofifa_id", "player_url", "nation_position", "long_name", "dob", "potential", "value_eur", "wage_eur", "international_reputation", "body_type", "real_face", "release_clause_eur", "player_tags", "loaned_from", "joined", "contract_valid_until", "height_cm", "weak_foot", "skill_moves", "work_rate", "team_jersey_number", "weight_kg", "age", "player_positions", "preferred_foot", "nation_jersey_number"], axis = 1)
fifa_19 = fifa_19.iloc[:, range(0, 17)]
fifa_19 = fifa_19.rename(columns = {"short_name": "name", "team_position": "position", "physic": "physical"})

fifa_20 = fifa_20.drop(["sofifa_id", "player_url", "nation_position", "long_name", "dob", "potential", "value_eur", "wage_eur", "international_reputation", "body_type", "real_face", "release_clause_eur", "player_tags", "loaned_from", "joined", "contract_valid_until", "height_cm", "weak_foot", "skill_moves", "work_rate", "team_jersey_number", "weight_kg", "age", "player_positions", "preferred_foot", "nation_jersey_number"], axis = 1)
fifa_20 = fifa_20.iloc[:, range(0, 17)]
fifa_20 = fifa_20.rename(columns = {"short_name": "name", "team_position": "position", "physic": "physical"})

In [127]:
fifa_15.loc[(fifa_15.position == "LCM") | (fifa_15.position == "RCM"), "position"] = "CM"
fifa_15.loc[(fifa_15.position == "LCB") | (fifa_15.position == "RCB"), "position"] = "CB"
fifa_15.loc[(fifa_15.position == "RDM") | (fifa_15.position == "LDM"), "position"] = "CDM"
fifa_15.loc[(fifa_15.position == "RAM") | (fifa_15.position == "LAM"), "position"] = "CAM"
fifa_15.loc[(fifa_15.position == "LS") | (fifa_15.position == "RS"), "position"] = "ST"

fifa_16.loc[(fifa_16.position == "LCM") | (fifa_16.position == "RCM"), "position"] = "CM"
fifa_16.loc[(fifa_16.position == "LCB") | (fifa_16.position == "RCB"), "position"] = "CB"
fifa_16.loc[(fifa_16.position == "RDM") | (fifa_16.position == "LDM"), "position"] = "CDM"
fifa_16.loc[(fifa_16.position == "RAM") | (fifa_16.position == "LAM"), "position"] = "CAM"
fifa_16.loc[(fifa_16.position == "LS") | (fifa_16.position == "RS"), "position"] = "ST"

fifa_17.loc[(fifa_17.position == "LCM") | (fifa_17.position == "RCM"), "position"] = "CM"
fifa_17.loc[(fifa_17.position == "LCB") | (fifa_17.position == "RCB"), "position"] = "CB"
fifa_17.loc[(fifa_17.position == "RDM") | (fifa_17.position == "LDM"), "position"] = "CDM"
fifa_17.loc[(fifa_17.position == "RAM") | (fifa_17.position == "LAM"), "position"] = "CAM"
fifa_17.loc[(fifa_17.position == "LS") | (fifa_17.position == "RS"), "position"] = "ST"

fifa_18.loc[(fifa_18.position == "LCM") | (fifa_18.position == "RCM"), "position"] = "CM"
fifa_18.loc[(fifa_18.position == "LCB") | (fifa_18.position == "RCB"), "position"] = "CB"
fifa_18.loc[(fifa_18.position == "RDM") | (fifa_18.position == "LDM"), "position"] = "CDM"
fifa_18.loc[(fifa_18.position == "RAM") | (fifa_18.position == "LAM"), "position"] = "CAM"
fifa_18.loc[(fifa_18.position == "LS") | (fifa_18.position == "RS"), "position"] = "ST"

fifa_19.loc[(fifa_19.position == "LCM") | (fifa_19.position == "RCM"), "position"] = "CM"
fifa_19.loc[(fifa_19.position == "LCB") | (fifa_19.position == "RCB"), "position"] = "CB"
fifa_19.loc[(fifa_19.position == "RDM") | (fifa_19.position == "LDM"), "position"] = "CDM"
fifa_19.loc[(fifa_19.position == "RAM") | (fifa_19.position == "LAM"), "position"] = "CAM"
fifa_19.loc[(fifa_19.position == "LS") | (fifa_19.position == "RS"), "position"] = "ST"

fifa_20.loc[(fifa_20.position == "LCM") | (fifa_20.position == "RCM"), "position"] = "CM"
fifa_20.loc[(fifa_20.position == "LCB") | (fifa_20.position == "RCB"), "position"] = "CB"
fifa_20.loc[(fifa_20.position == "RDM") | (fifa_20.position == "LDM"), "position"] = "CDM"
fifa_20.loc[(fifa_20.position == "RAM") | (fifa_20.position == "LAM"), "position"] = "CAM"
fifa_20.loc[(fifa_20.position == "LS") | (fifa_20.position == "RS"), "position"] = "ST"

In [128]:
def starting_11_checker(fifa, team):
    if (fifa.loc[(fifa.club == team) & (fifa.position != "SUB") & (fifa.position != "RES")]["position"].count()) == 11:
        return True
    else:
        return False

Premier League Cleaning

In [129]:
prem_1819 = pd.read_csv("./England/prem1819.csv")
prem_1718 = pd.read_csv("./England/prem1718.csv")
prem_1617 = pd.read_csv("./England/prem1617.csv")
prem_1516 = pd.read_csv("./England/prem1516.csv")
prem_1415 = pd.read_csv("./England/prem1415.csv") # row 382 blank
prem_1314 = pd.read_csv("./England/prem1314.csv")

prem_1819 = prem_1819.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
prem_1819["Fifa"] = "fifa_20"
prem_1718 = prem_1718.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
prem_1718["Fifa"] = "fifa_19"
prem_1617 = prem_1617.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
prem_1617["Fifa"] = "fifa_18"
prem_1516 = prem_1516.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
prem_1516["Fifa"] = "fifa_17"
prem_1415 = prem_1415.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
prem_1415["Fifa"] = "fifa_16"
prem_1314 = prem_1314.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
prem_1314["Fifa"] = "fifa_15"

prem = pd.concat([prem_1819, prem_1718, prem_1617, prem_1516, prem_1415, prem_1314])
prem = prem.rename(columns = {"FTR": "winner", "HomeTeam": "home", "AwayTeam": "away"})

prem.loc[(prem.home == "Man United"), "home"] = "Manchester United"
prem.loc[(prem.away == "Man United"), "away"] = "Manchester United"
prem.loc[(prem.home == "Huddersfield"), "home"] = "Huddersfield Town"
prem.loc[(prem.away == "Huddersfield"), "away"] = "Huddersfield Town"
prem.loc[(prem.home == "Newcastle"), "home"] = "Newcastle United"
prem.loc[(prem.away == "Newcastle"), "away"] = "Newcastle United"
prem.loc[(prem.home == "Wolves"), "home"] = "Wolverhampton Wanderers"
prem.loc[(prem.away == "Wolves"), "away"] = "Wolverhampton Wanderers"
prem.loc[(prem.home == "Cardiff"), "home"] =  "Cardiff City"
prem.loc[(prem.away == "Cardiff"), "away"] = "Cardiff City"
prem.loc[(prem.home == "Leicester"), "home"] = "Leicester City"
prem.loc[(prem.away == "Leicester"), "away"] = "Leicester City"
prem.loc[(prem.home == "Tottenham"), "home"] = "Tottenham Hotspur"
prem.loc[(prem.away == "Tottenham"), "away"] = "Tottenham Hotspur"
prem.loc[(prem.home == "West Ham"), "home"] = "West Ham United"
prem.loc[(prem.away == "West Ham"), "away"] = "West Ham United"
prem.loc[(prem.home == "Brighton"), "home"] = "Brighton & Hove Albion"
prem.loc[(prem.away == "Brighton"), "away"] = "Brighton & Hove Albion"
prem.loc[(prem.home == "Man City"), "home"] = "Manchester City"
prem.loc[(prem.away == "Man City"), "away"] = "Manchester City"
prem.loc[(prem.home == "West Brom"), "home"] = "West Bromwich Albion"
prem.loc[(prem.away == "West Brom"), "away"] = "West Bromwich Albion"
prem.loc[(prem.home == "Stoke"), "home"] = "Stoke City"
prem.loc[(prem.away == "Stoke"), "away"] = "Stoke City"
prem.loc[(prem.home == "Swansea"), "home"] = "Swansea City"
prem.loc[(prem.away == "Swansea"), "away"] = "Swansea City"
prem.loc[(prem.home == "Hull"), "home"] = "Hull City"
prem.loc[(prem.away == "Hull"), "away"] = "Hull City"
prem.loc[(prem.home == "Norwich"), "home"] = "Norwich City"
prem.loc[(prem.away == "Norwich"), "away"] = "Norwich City"
prem.loc[(prem.home == "QPR"), "home"] = "Queens Park Rangers"
prem.loc[(prem.away == "QPR"), "away"] = "Queens Park Rangers"

fifa_15.loc[fifa_15.name == "C. Jenkinson", "position"] = "RB"
fifa_16.loc[fifa_16.name == "W. Rooney", "position"] = "ST"
fifa_16.loc[fifa_16.name == "B. Schweinsteiger", "position"] = "CAM"
fifa_16.loc[fifa_16.name == "D. Burn", "position"] = "CB"
fifa_16.loc[fifa_16.name == "R. Bertrand", "position"] = "CB"
fifa_16.loc[fifa_16.name == "C. Eriksen", "position"] = "CAM"
fifa_16.loc[fifa_16.name == "A. Johnson", "position"] = "LM"
fifa_16.loc[fifa_16.name == "J. Okore", "position"] = "CB"

prem_teams = prem.home.unique()
prem_teams

array(['Manchester United', 'Bournemouth', 'Fulham', 'Huddersfield Town',
       'Newcastle United', 'Watford', 'Wolverhampton Wanderers',
       'Arsenal', 'Liverpool', 'Southampton', 'Cardiff City', 'Chelsea',
       'Everton', 'Leicester City', 'Tottenham Hotspur',
       'West Ham United', 'Brighton & Hove Albion', 'Burnley',
       'Manchester City', 'Crystal Palace', 'West Bromwich Albion',
       'Stoke City', 'Swansea City', 'Hull City', 'Middlesbrough',
       'Sunderland', 'Norwich City', 'Aston Villa', 'Queens Park Rangers'],
      dtype=object)

In [130]:
for i in range(29):
    if starting_11_checker(fifa_15, prem_teams[i]) != True:
        print(prem_teams[i])
for i in range(29):
    if starting_11_checker(fifa_16, prem_teams[i]) != True:
        print(prem_teams[i])
for i in range(29):
    if starting_11_checker(fifa_17, prem_teams[i]) != True:
        print(prem_teams[i])
for i in range(29):
    if starting_11_checker(fifa_18, prem_teams[i]) != True:
        print(prem_teams[i])
for i in range(29):
    if starting_11_checker(fifa_19, prem_teams[i]) != True:
        print(prem_teams[i])
for i in range(29):
    if starting_11_checker(fifa_20, prem_teams[i]) != True:
        print(prem_teams[i])

Italian League Cleaning

In [131]:
serie_a_1819 = pd.read_csv("./Italy/ita1819.csv")
serie_a_1718 = pd.read_csv("./Italy/ita1718.csv")
serie_a_1617 = pd.read_csv("./Italy/ita1617.csv")
serie_a_1516 = pd.read_csv("./Italy/ita1516.csv") # row 382 blank
serie_a_1314 = pd.read_csv("./Italy/ita1314.csv")

serie_a_1819 = serie_a_1819.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
serie_a_1819["Fifa"] = "fifa_20"
serie_a_1718 = serie_a_1718.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
serie_a_1718["Fifa"] = "fifa_19"
serie_a_1617 = serie_a_1617.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
serie_a_1617["Fifa"] = "fifa_18"
serie_a_1516 = serie_a_1516.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
serie_a_1516["Fifa"] = "fifa_17"
serie_a_1314 = serie_a_1314.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
serie_a_1314["Fifa"] = "fifa_15"

serie_a = pd.concat([serie_a_1819, serie_a_1718, serie_a_1617, serie_a_1516, serie_a_1314])
serie_a = serie_a.rename(columns = {"FTR": "winner", "HomeTeam": "home", "AwayTeam": "away"})

serie_a.loc[(serie_a.home == "Chievo"), "home"] = "Chievo Verona"
serie_a.loc[(serie_a.away == "Chievo"), "away"] = "Chievo Verona"
serie_a.loc[(serie_a.home == "Verona"), "home"] = "Hellas Verona"
serie_a.loc[(serie_a.away == "Verona"), "away"] = "Hellas Verona"
serie_a.loc[(serie_a.home == "Spal"), "home"] = "SPAL"
serie_a.loc[(serie_a.away == "Spal"), "away"] = "SPAL"

fifa_16.loc[fifa_16.name == "S. Khedira", "position"] = "CM"
fifa_16.loc[fifa_16.name == "A. Ljajić", "position"] = "CAM"
fifa_16.loc[fifa_16.name == "F. Guarín", "position"] = "CM"
fifa_16.loc[fifa_16.name == "R. Montolivo", "position"] = "CM"
fifa_16.loc[fifa_16.name == "R. Fiamozzi", "position"] = "CB"
fifa_16.loc[fifa_16.name == "G. Lapadula", "position"] = "ST"
fifa_16.loc[fifa_16.name == "I. Bubnjić", "position"] = "CB"
fifa_16.loc[(fifa_16.name == "Emerson") & (fifa_16.club == "Livorno"), "position"] = "CB"
fifa_19.loc[(fifa_19.name == "M. Gasperi") & (fifa_19.club == "Cesena"), "position"] = "CM"
fifa_16.loc[(fifa_16.name == "M. Gasperi") & (fifa_16.club == "Cesena"), "position"] = "SUB"

serie_a_teams = serie_a.home.unique()
serie_a_teams

array(['Chievo Verona', 'Lazio', 'Bologna', 'Empoli', 'Parma', 'Sassuolo',
       'Torino', 'Atalanta', 'Juventus', 'Napoli', 'Cagliari',
       'Fiorentina', 'Frosinone', 'Genoa', 'Inter', 'SPAL', 'Udinese',
       'Roma', 'Milan', 'Sampdoria', 'Hellas Verona', 'Crotone',
       'Benevento', 'Palermo', 'Pescara', 'Carpi', 'Livorno', 'Catania'],
      dtype=object)

In [133]:
for i in range(28):
    if (serie_a_teams[i] != "SPAL") and (serie_a_teams[i] != "Benevento"):
        if starting_11_checker(fifa_15, serie_a_teams[i]) != True:
            print(serie_a_teams[i])
for i in range(28):
    if (serie_a_teams[i] != "Catania") and (serie_a_teams[i] != "Parma") and (serie_a_teams[i] != "Livorno"):
        if starting_11_checker(fifa_17, serie_a_teams[i]) != True:
            print(serie_a_teams[i])
for i in range(28):
    if (serie_a_teams[i] != "Catania") and (serie_a_teams[i] != "Livorno"):
        if starting_11_checker(fifa_18, serie_a_teams[i]) != True:
            print(serie_a_teams[i])
for i in range(28):
    if (serie_a_teams[i] != "Catania"):
        if starting_11_checker(fifa_19, serie_a_teams[i]) != True:
            print(serie_a_teams[i])
for i in range(28):
    if (serie_a_teams[i] != "Palermo") and (serie_a_teams[i] != "Catania") and (serie_a_teams[i] != "Cesena") and (serie_a_teams[i] != "Carpi"):
        if starting_11_checker(fifa_20, serie_a_teams[i]) != True:
            print(serie_a_teams[i])

French League Cleaning

In [134]:
league_one_1819 = pd.read_csv("./France/fra1819.csv")
league_one_1718 = pd.read_csv("./France/fra1718.csv")

league_one_1819 = league_one_1819.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
league_one_1819["Fifa"] = "fifa_20"
league_one_1718 = league_one_1718.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
league_one_1718["Fifa"] = "fifa_19"

league_one = pd.concat([league_one_1819, league_one_1718])
league_one = league_one.rename(columns = {"FTR": "winner", "HomeTeam": "home", "AwayTeam": "away"})

league_one.loc[(league_one.home == "Marseille"), "home"] = "Olympique de Marseille"
league_one.loc[(league_one.away == "Marseille"), "away"] = "Olympique de Marseille"
league_one.loc[(league_one.home == "Angers"), "home"] = "Angers SCO"
league_one.loc[(league_one.away == "Angers"), "away"] = "Angers SCO"
league_one.loc[(league_one.home == "Lille"), "home"] = "LOSC Lille"
league_one.loc[(league_one.away == "Lille"), "away"] = "LOSC Lille"
league_one.loc[(league_one.home == "Montpellier"), "home"] = "Montpellier HSC"
league_one.loc[(league_one.away == "Montpellier"), "away"] = "Montpellier HSC"
league_one.loc[(league_one.home == "Nantes"), "home"] = "FC Nantes"
league_one.loc[(league_one.away == "Nantes"), "away"] = "FC Nantes"
league_one.loc[(league_one.home == "Nice"), "home"] = "OGC Nice"
league_one.loc[(league_one.away == "Nice"), "away"] = "OGC Nice"
league_one.loc[(league_one.home == "St Etienne"), "home"] = "AS Saint-Étienne"
league_one.loc[(league_one.away == "St Etienne"), "away"] = "AS Saint-Étienne"
league_one.loc[(league_one.home == "Bordeaux"), "home"] = "FC Girondins de Bordeaux"
league_one.loc[(league_one.away == "Bordeaux"), "away"] = "FC Girondins de Bordeaux"
league_one.loc[(league_one.home == "Lyon"), "home"] = "Olympique Lyonnais"
league_one.loc[(league_one.away == "Lyon"), "away"] = "Olympique Lyonnais"
league_one.loc[(league_one.home == "Paris SG"), "home"] = "Paris Saint-Germain"
league_one.loc[(league_one.away == "Paris SG"), "away"] = "Paris Saint-Germain"
league_one.loc[(league_one.home == "Reims"), "home"] = "Stade de Reims"
league_one.loc[(league_one.away == "Reims"), "away"] = "Stade de Reims"
league_one.loc[(league_one.home == "Amiens"), "home"] = "Amiens SC"
league_one.loc[(league_one.away == "Amiens"), "away"] = "Amiens SC"
league_one.loc[(league_one.home == "Caen"), "home"] = "Stade Malherbe Caen"
league_one.loc[(league_one.away == "Caen"), "away"] = "Stade Malherbe Caen"
league_one.loc[(league_one.home == "Dijon"), "home"] = "Dijon FCO"
league_one.loc[(league_one.away == "Dijon"), "away"] = "Dijon FCO"
league_one.loc[(league_one.home == "Guingamp"), "home"] = "En Avant de Guingamp"
league_one.loc[(league_one.away == "Guingamp"), "away"] = "En Avant de Guingamp"
league_one.loc[(league_one.home == "Monaco"), "home"] = "AS Monaco"
league_one.loc[(league_one.away == "Monaco"), "away"] = "AS Monaco"
league_one.loc[(league_one.home == "Rennes"), "home"] = "Stade Rennais FC"
league_one.loc[(league_one.away == "Rennes"), "away"] = "Stade Rennais FC"
league_one.loc[(league_one.home == "Nimes"), "home"] = "Nîmes Olympique"
league_one.loc[(league_one.away == "Nimes"), "away"] = "Nîmes Olympique"
league_one.loc[(league_one.home == "Strasbourg"), "home"] = "RC Strasbourg Alsace"
league_one.loc[(league_one.away == "Strasbourg"), "away"] = "RC Strasbourg Alsace"
league_one.loc[(league_one.home == "Toulouse"), "home"] = "Toulouse Football Club"
league_one.loc[(league_one.away == "Toulouse"), "away"] = "Toulouse Football Club"
league_one.loc[(league_one.home == "Metz"), "home"] = "FC Metz"
league_one.loc[(league_one.away == "Metz"), "away"] = "FC Metz"
league_one.loc[(league_one.home == "Troyes"), "home"] = "ESTAC Troyes"
league_one.loc[(league_one.away == "Troyes"), "away"] = "ESTAC Troyes"
league_one.loc[(league_one.home == "Nancy"), "home"] = "AS Nancy Lorraine"
league_one.loc[(league_one.away == "Nancy"), "away"] = "AS Nancy Lorraine"
league_one.loc[(league_one.home == "Lorient"), "home"] = "FC Lorient"
league_one.loc[(league_one.away == "Lorient"), "away"] = "FC Lorient"
league_one.loc[(league_one.home == "Ajaccio GFCO"), "home"] = "GFC Ajaccio"
league_one.loc[(league_one.away == "Ajaccio GFCO"), "away"] = "GFC Ajaccio"
league_one.loc[(league_one.home == "Lens"), "home"] = "Racing Club de Lens"
league_one.loc[(league_one.away == "Lens"), "away"] = "Racing Club de Lens"
league_one.loc[(league_one.home == "Valenciennes"), "home"] = "Valenciennes FC"
league_one.loc[(league_one.away == "Valenciennes"), "away"] = "Valenciennes FC"
league_one.loc[(league_one.home == "Ajaccio"), "home"] = "AC Ajaccio"
league_one.loc[(league_one.away == "Ajaccio"), "away"] = "AC Ajaccio"
league_one.loc[(league_one.home == "Sochaux"), "home"] = "FC Sochaux-Montbéliard"
league_one.loc[(league_one.away == "Sochaux"), "away"] = "FC Sochaux-Montbéliard"

league_one_teams = league_one.home.unique()
league_one_teams = league_one_teams.tolist()
# league_one_teams.remove("Bastia")
# league_one_teams.remove('Evian Thonon Gaillard')
league_one_teams = np.asarray(league_one_teams)
league_one_teams

array(['Olympique de Marseille', 'Angers SCO', 'LOSC Lille',
       'Montpellier HSC', 'FC Nantes', 'OGC Nice', 'AS Saint-Étienne',
       'FC Girondins de Bordeaux', 'Olympique Lyonnais',
       'Paris Saint-Germain', 'Stade de Reims', 'Amiens SC',
       'Stade Malherbe Caen', 'Dijon FCO', 'En Avant de Guingamp',
       'AS Monaco', 'Stade Rennais FC', 'Nîmes Olympique',
       'RC Strasbourg Alsace', 'Toulouse Football Club', 'FC Metz',
       'ESTAC Troyes'], dtype='<U24')

In [135]:
for i in range(22):
    if (league_one_teams[i] != "GFC Ajaccio"):
        if starting_11_checker(fifa_20, league_one_teams[i]) != True:
            print(league_one_teams[i])
for i in range(22):
    if starting_11_checker(fifa_19, league_one_teams[i]) != True:
        print(league_one_teams[i])

Bundesliga Cleaning

In [136]:
bundesliga_1819 = pd.read_csv("./Germany/ger1819.csv")
bundesliga_1718 = pd.read_csv("./Germany/ger1718.csv")
bundesliga_1617 = pd.read_csv("./Germany/ger1617.csv")
bundesliga_1516 = pd.read_csv("./Germany/ger1516.csv")
bundesliga_1415 = pd.read_csv("./Germany/ger1415.csv")
bundesliga_1314 = pd.read_csv("./Germany/ger1314.csv")

bundesliga_1819 = bundesliga_1819.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
bundesliga_1819["Fifa"] = "fifa_20"
bundesliga_1718 = bundesliga_1718.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
bundesliga_1718["Fifa"] = "fifa_19"
bundesliga_1617 = bundesliga_1617.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
bundesliga_1617["Fifa"] = "fifa_18"
bundesliga_1516 = bundesliga_1516.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
bundesliga_1516["Fifa"] = "fifa_17"
bundesliga_1415 = bundesliga_1415.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
bundesliga_1415["Fifa"] = "fifa_16"
bundesliga_1314 = bundesliga_1314.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
bundesliga_1314["Fifa"] = "fifa_15"

bundesliga = pd.concat([bundesliga_1819, bundesliga_1718, bundesliga_1617, bundesliga_1516, bundesliga_1415, bundesliga_1314])
bundesliga = bundesliga.rename(columns = {"FTR": "winner", "HomeTeam": "home", "AwayTeam": "away"})

bundesliga.loc[(bundesliga.home == "Bayern Munich"), "home"] = "FC Bayern München"
bundesliga.loc[(bundesliga.away == "Bayern Munich"), "away"] = "FC Bayern München"
bundesliga.loc[(bundesliga.home == "Fortuna Dusseldorf"), "home"] = "Fortuna Düsseldorf"
bundesliga.loc[(bundesliga.away == "Fortuna Dusseldorf"), "away"] = "Fortuna Düsseldorf"
bundesliga.loc[(bundesliga.home == "Freiburg"), "home"] = "SC Freiburg"
bundesliga.loc[(bundesliga.away == "Freiburg"), "away"] = "SC Freiburg"
bundesliga.loc[(bundesliga.home == "Hertha"), "home"] = "Hertha BSC"
bundesliga.loc[(bundesliga.away == "Hertha"), "away"] = "Hertha BSC"
bundesliga.loc[(bundesliga.home == "M'gladbach"), "home"] = "Borussia Mönchengladbach"
bundesliga.loc[(bundesliga.away == "M'gladbach"), "away"] = "Borussia Mönchengladbach"
bundesliga.loc[(bundesliga.home == "Freiburg"), "home"] = "SC Freiburg"
bundesliga.loc[(bundesliga.away == "Freiburg"), "away"] = "SC Freiburg"
bundesliga.loc[(bundesliga.home == "Werder Bremen"), "home"] = "SV Werder Bremen"
bundesliga.loc[(bundesliga.away == "Werder Bremen"), "away"] = "SV Werder Bremen"
bundesliga.loc[(bundesliga.home == "Wolfsburg"), "home"] = "VfL Wolfsburg"
bundesliga.loc[(bundesliga.away == "Wolfsburg"), "away"] = "VfL Wolfsburg"
bundesliga.loc[(bundesliga.home == "Dortmund"), "home"] = "Borussia Dortmund"
bundesliga.loc[(bundesliga.away == "Dortmund"), "away"] = "Borussia Dortmund"
bundesliga.loc[(bundesliga.home == "Mainz"), "home"] = "1. FSV Mainz 05"
bundesliga.loc[(bundesliga.away == "Mainz"), "away"] = "1. FSV Mainz 05"
bundesliga.loc[(bundesliga.home == "Hannover"), "home"] = "Hannover 96"
bundesliga.loc[(bundesliga.away == "Hannover"), "away"] = "Hannover 96"
bundesliga.loc[(bundesliga.home == "Augsburg"), "home"] = "FC Augsburg"
bundesliga.loc[(bundesliga.away == "Augsburg"), "away"] = "FC Augsburg"
bundesliga.loc[(bundesliga.home == "Ein Frankfurt"), "home"] = "Eintracht Frankfurt"
bundesliga.loc[(bundesliga.away == "Ein Frankfurt"), "away"] = "Eintracht Frankfurt"
bundesliga.loc[(bundesliga.home == "Hoffenheim"), "home"] = "TSG 1899 Hoffenheim"
bundesliga.loc[(bundesliga.away == "Hoffenheim"), "away"] = "TSG 1899 Hoffenheim"
bundesliga.loc[(bundesliga.home == "Leverkusen"), "home"] = "Bayer 04 Leverkusen"
bundesliga.loc[(bundesliga.away == "Leverkusen"), "away"] = "Bayer 04 Leverkusen"
bundesliga.loc[(bundesliga.home == "Nurnberg"), "home"] = "1. FC Nürnberg"
bundesliga.loc[(bundesliga.away == "Nurnberg"), "away"] = "1. FC Nürnberg"
bundesliga.loc[(bundesliga.home == "Stuttgart"), "home"] = "VfB Stuttgart"
bundesliga.loc[(bundesliga.away == "Stuttgart"), "away"] = "VfB Stuttgart"
bundesliga.loc[(bundesliga.home == "Schalke 04"), "home"] = "FC Schalke 04"
bundesliga.loc[(bundesliga.away == "Schalke 04"), "away"] = "FC Schalke 04"
bundesliga.loc[(bundesliga.home == "Hamburg"), "home"] = "Hamburger SV"
bundesliga.loc[(bundesliga.away == "Hamburg"), "away"] = "Hamburger SV"
bundesliga.loc[(bundesliga.home == "FC Koln"), "home"] = "1. FC Köln"
bundesliga.loc[(bundesliga.away == "FC Koln"), "away"] = "1. FC Köln"
bundesliga.loc[(bundesliga.home == "Darmstadt"), "home"] = "SV Darmstadt 98"
bundesliga.loc[(bundesliga.away == "Darmstadt"), "away"] = "SV Darmstadt 98"
bundesliga.loc[(bundesliga.home == "Ingolstadt"), "home"] = "FC Ingolstadt 04"
bundesliga.loc[(bundesliga.away == "Ingolstadt"), "away"] = "FC Ingolstadt 04"
bundesliga.loc[(bundesliga.home == "Paderborn"), "home"] = "SC Paderborn 07"
bundesliga.loc[(bundesliga.away == "Paderborn"), "away"] = "SC Paderborn 07"
bundesliga.loc[(bundesliga.home == "Braunschweig"), "home"] = "Eintracht Braunschweig"
bundesliga.loc[(bundesliga.away == "Braunschweig"), "away"] = "Eintracht Braunschweig"

fifa_15.loc[fifa_15.name == "Koo Ja Cheol", "position"] = "CAM"
fifa_16.loc[fifa_16.name == "J. Guilavogui", "position"] = "CB"
fifa_16.loc[fifa_16.name == "N. Bendtner", "position"] = "ST"
fifa_16.loc[fifa_16.name == "H. Altintop", "position"] = "CAM"
fifa_16.loc[fifa_16.name == "A. Nukan", "position"] = "LB"
fifa_16.loc[fifa_16.name == "M. Díaz", "position"] = "CAM"
fifa_16.loc[fifa_16.name == "S. Lakić", "position"] = "ST"

bundesliga_teams = bundesliga.home.unique()
bundesliga_teams

array(['FC Bayern München', 'Fortuna Düsseldorf', 'SC Freiburg',
       'Hertha BSC', 'Borussia Mönchengladbach', 'SV Werder Bremen',
       'VfL Wolfsburg', 'Borussia Dortmund', '1. FSV Mainz 05',
       'Hannover 96', 'FC Augsburg', 'Eintracht Frankfurt',
       'TSG 1899 Hoffenheim', 'Bayer 04 Leverkusen', '1. FC Nürnberg',
       'VfB Stuttgart', 'RB Leipzig', 'FC Schalke 04', 'Hamburger SV',
       '1. FC Köln', 'SV Darmstadt 98', 'FC Ingolstadt 04',
       'SC Paderborn 07', 'Eintracht Braunschweig'], dtype=object)

In [137]:
for i in range(24):
    if starting_11_checker(fifa_15, bundesliga_teams[i]) != True:
        print(bundesliga_teams[i])
for i in range(24):
    if starting_11_checker(fifa_16, bundesliga_teams[i]) != True:
        print(bundesliga_teams[i])
for i in range(24):
    if (bundesliga_teams[i] != "SC Paderborn 07"):
        if starting_11_checker(fifa_17, bundesliga_teams[i]) != True:
            print(bundesliga_teams[i])
for i in range(24):
        if starting_11_checker(fifa_18, bundesliga_teams[i]) != True:
            print(bundesliga_teams[i])
for i in range(24):
        if starting_11_checker(fifa_19, bundesliga_teams[i]) != True:
            print(bundesliga_teams[i])
for i in range(24):
        if starting_11_checker(fifa_20, bundesliga_teams[i]) != True:
            print(bundesliga_teams[i])

La Liga Cleaning

In [138]:
la_liga_1819 = pd.read_csv("./Spain/spain1819.csv")
la_liga_1718 = pd.read_csv("./Spain/spain1718.csv")
la_liga_1617 = pd.read_csv("./Spain/spain1617.csv")
la_liga_1516 = pd.read_csv("./Spain/spain1516.csv")
la_liga_1415 = pd.read_csv("./Spain/spain1415.csv")
la_liga_1314 = pd.read_csv("./Spain/spain1314.csv")

la_liga_1819 = la_liga_1819.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
la_liga_1819["Fifa"] = "fifa_20"
la_liga_1718 = la_liga_1718.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
la_liga_1718["Fifa"] = "fifa_19"
la_liga_1617 = la_liga_1617.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
la_liga_1617["Fifa"] = "fifa_18"
la_liga_1516 = la_liga_1516.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
la_liga_1516["Fifa"] = "fifa_17"
la_liga_1415 = la_liga_1415.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
la_liga_1415["Fifa"] = "fifa_16"
la_liga_1314 = la_liga_1314.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTR"]]
la_liga_1314["Fifa"] = "fifa_15"

la_liga = pd.concat([la_liga_1819, la_liga_1718, la_liga_1617, la_liga_1516, la_liga_1415, la_liga_1314])
la_liga = la_liga.rename(columns = {"FTR": "winner", "HomeTeam": "home", "AwayTeam": "away"})

la_liga.loc[(la_liga.home == "Betis"), "home"] = "Real Betis"
la_liga.loc[(la_liga.away == "Betis"), "away"] = "Real Betis"
la_liga.loc[(la_liga.home == "Girona"), "home"] = "Girona FC"
la_liga.loc[(la_liga.away == "Girona"), "away"] = "Girona FC"
la_liga.loc[(la_liga.home == "Barcelona"), "home"] = "FC Barcelona"
la_liga.loc[(la_liga.away == "Barcelona"), "away"] = "FC Barcelona"
la_liga.loc[(la_liga.home == "Celta"), "home"] = "RC Celta"
la_liga.loc[(la_liga.away == "Celta"), "away"] = "RC Celta"
la_liga.loc[(la_liga.home == "Villarreal"), "home"] = "Villarreal CF"
la_liga.loc[(la_liga.away == "Villarreal"), "away"] = "Villarreal CF"
la_liga.loc[(la_liga.home == "Eibar"), "home"] = "SD Eibar"
la_liga.loc[(la_liga.away == "Eibar"), "away"] = "SD Eibar"
la_liga.loc[(la_liga.home == "Vallecano"), "home"] = "Rayo Vallecano"
la_liga.loc[(la_liga.away == "Vallecano"), "away"] = "Rayo Vallecano"
la_liga.loc[(la_liga.home == "Ath Bilbao"), "home"] = "Athletic Club de Bilbao"
la_liga.loc[(la_liga.away == "Ath Bilbao"), "away"] = "Athletic Club de Bilbao"
la_liga.loc[(la_liga.home == "Valencia"), "home"] = "Valencia CF"
la_liga.loc[(la_liga.away == "Valencia"), "away"] = "Valencia CF"
la_liga.loc[(la_liga.home == "Getafe"), "home"] = "Getafe CF"
la_liga.loc[(la_liga.away == "Getafe"), "away"] = "Getafe CF"
la_liga.loc[(la_liga.home == "Leganes"), "home"] = "CD Leganés"
la_liga.loc[(la_liga.away == "Leganes"), "away"] = "CD Leganés"
la_liga.loc[(la_liga.home == "Alaves"), "home"] = "Deportivo Alavés"
la_liga.loc[(la_liga.away == "Alaves"), "away"] = "Deportivo Alavés"
la_liga.loc[(la_liga.home == "Leganes"), "home"] = "CD Leganés"
la_liga.loc[(la_liga.away == "Leganes"), "away"] = "CD Leganés"
la_liga.loc[(la_liga.home == "Ath Madrid"), "home"] = "Atlético Madrid"
la_liga.loc[(la_liga.away == "Ath Madrid"), "away"] = "Atlético Madrid"
la_liga.loc[(la_liga.home == "Valladolid"), "home"] = "Real Valladolid CF"
la_liga.loc[(la_liga.away == "Valladolid"), "away"] = "Real Valladolid CF"
la_liga.loc[(la_liga.home == "Espanol"), "home"] = "RCD Espanyol"
la_liga.loc[(la_liga.away == "Espanol"), "away"] = "RCD Espanyol"
la_liga.loc[(la_liga.home == "Sevilla"), "home"] = "Sevilla FC"
la_liga.loc[(la_liga.away == "Sevilla"), "away"] = "Sevilla FC"
la_liga.loc[(la_liga.home == "Levante"), "home"] = "Levante UD"
la_liga.loc[(la_liga.away == "Levante"), "away"] = "Levante UD"
la_liga.loc[(la_liga.home == "Huesca"), "home"] = "SD Huesca"
la_liga.loc[(la_liga.away == "Huesca"), "away"] = "SD Huesca"
la_liga.loc[(la_liga.home == "Sociedad"), "home"] = "Real Sociedad"
la_liga.loc[(la_liga.away == "Sociedad"), "away"] = "Real Sociedad"
la_liga.loc[(la_liga.home == "La Coruna"), "home"] = "Deportivo de La Coruña"
la_liga.loc[(la_liga.away == "La Coruna"), "away"] = "Deportivo de La Coruña"
la_liga.loc[(la_liga.home == "Malaga"), "home"] = "Málaga CF"
la_liga.loc[(la_liga.away == "Malaga"), "away"] = "Málaga CF"
la_liga.loc[(la_liga.home == "Las Palmas"), "home"] = "UD Las Palmas"
la_liga.loc[(la_liga.away == "Las Palmas"), "away"] = "UD Las Palmas"
la_liga.loc[(la_liga.home == "Granada"), "home"] = "Granada CF"
la_liga.loc[(la_liga.away == "Granada"), "away"] = "Granada CF"
la_liga.loc[(la_liga.home == "Sp Gijon"), "home"] = "Real Sporting de Gijón"
la_liga.loc[(la_liga.away == "Sp Gijon"), "away"] = "Real Sporting de Gijón"
la_liga.loc[(la_liga.home == "Osasuna"), "home"] = "CA Osasuna"
la_liga.loc[(la_liga.away == "Osasuna"), "away"] = "CA Osasuna"
la_liga.loc[(la_liga.home == "Almeria"), "home"] = "UD Almería"
la_liga.loc[(la_liga.away == "Almeria"), "away"] = "UD Almería"
la_liga.loc[(la_liga.home == "Cordoba"), "home"] = "Córdoba CF"
la_liga.loc[(la_liga.away == "Cordoba"), "away"] = "Córdoba CF"
la_liga.loc[(la_liga.home == "Elche"), "home"] = "Elche CF"
la_liga.loc[(la_liga.away == "Elche"), "away"] = "Elche CF"

fifa_15.loc[fifa_15.name == "L. Suárez", "position"] = "ST"
fifa_16.loc[fifa_16.name == "Dani Ceballos", "position"] = "RM"
fifa_16.loc[fifa_16.name == "A. Turan", "position"] = "CM"
fifa_16.loc[fifa_16.name == "Ibai Gómez", "position"] = "CM"
fifa_16.loc[(fifa_16.name == "Danilo") & (fifa_16.club == "Valencia CF"), "position"] = "CB"
fifa_16.loc[(fifa_16.name == "Rodri") & (fifa_16.club == "Real Valladolid CF"), "position"] = "ST"
fifa_16.loc[(fifa_16.name == "J. Rodríguez") & (fifa_16.club == "Deportivo de La Coruña"), "position"] = "LM"
fifa_17.loc[fifa_17.name == "A. Turan", "position"] = "CM"
fifa_18.loc[fifa_18.name == "A. Turan", "position"] = "CM"

la_liga_teams = la_liga.home.unique()
la_liga_teams

array(['Real Betis', 'Girona FC', 'FC Barcelona', 'RC Celta',
       'Villarreal CF', 'SD Eibar', 'Real Madrid', 'Rayo Vallecano',
       'Athletic Club de Bilbao', 'Valencia CF', 'Getafe CF',
       'CD Leganés', 'Deportivo Alavés', 'Atlético Madrid',
       'Real Valladolid CF', 'RCD Espanyol', 'Sevilla FC', 'Levante UD',
       'SD Huesca', 'Real Sociedad', 'Deportivo de La Coruña',
       'Málaga CF', 'UD Las Palmas', 'Granada CF',
       'Real Sporting de Gijón', 'CA Osasuna', 'UD Almería', 'Córdoba CF',
       'Elche CF'], dtype=object)

In [139]:
for i in range(29):
    if (la_liga_teams[i] != "SD Huesca"):
        if starting_11_checker(fifa_15, la_liga_teams[i]) != True:
            print(la_liga_teams[i])
for i in range(29):
      if starting_11_checker(fifa_16, la_liga_teams[i]) != True:
            print(la_liga_teams[i])
for i in range(29):
      if starting_11_checker(fifa_17, la_liga_teams[i]) != True:
            print(la_liga_teams[i])
for i in range(29):
    if (la_liga_teams[i] != "Elche CF"):
        if starting_11_checker(fifa_18, la_liga_teams[i]) != True:
            print(la_liga_teams[i])
for i in range(29):
      if starting_11_checker(fifa_19, la_liga_teams[i]) != True:
            print(la_liga_teams[i])
for i in range(29):
    if (la_liga_teams[i] != "Córdoba CF"):
        if starting_11_checker(fifa_20, la_liga_teams[i]) != True:
            print(la_liga_teams[i])

In [140]:
def soccer_winner_predictor(row):
    team1 = row['home']
    team2 = row['away']
    fifa = row['Fifa']
    if (fifa == "fifa_15"):
        fifa = fifa_15
    elif (fifa == "fifa_16"):
        fifa = fifa_16
    elif (fifa == "fifa_17"):
        fifa = fifa_17
    elif (fifa == "fifa_18"):
        fifa = fifa_18
    elif (fifa == "fifa_19"):
        fifa = fifa_19
    elif (fifa == "fifa_20"):
        fifa = fifa_20
    else:
        return False
    
    t1 = fifa.loc[fifa["club"] == team1]
    t2 = fifa.loc[fifa["club"] == team2]
    
    t1_forwards = t1.loc[t1.position.isin(forward)]
    t1_mids = t1.loc[t1.position.isin(midfielder)]
    t1_defenders = t1.loc[t1.position.isin(defender)]
    t1_gk = t1.loc[t1.position == "GK"]
    t1_sub = t1.loc[t1.position == "SUB"].head(3)

    t2_forwards = t2.loc[t2.position.isin(forward)]
    t2_mids = t2.loc[t2.position.isin(midfielder)]
    t2_defenders = t2.loc[t2.position.isin(defender)]
    t2_gk = t2.loc[t2.position == "GK"]
    t2_sub = t2.loc[t2.position == "SUB"].head(3)

    t1_gk_avg = np.mean(t1_gk.overall) # t1 gk overall
    t2_gk_avg = np.mean(t2_gk.overall) # t2 gk overall

    t1_sub_avg = .75 * np.mean(t1_sub.overall) # t1 sub avg
    t2_sub_avg = .75 * np.mean(t2_sub.overall) # t2 sub avg
    
    t1_mid_avg = np.mean(t1_mids.overall) # t1 mid avg
    t2_mid_avg = np.mean(t2_mids.overall) # t2 mid avg
        
    t1_forward_avg = np.mean(t1_forwards.overall) # t1 frwd shooting avg
    t2_forward_avg = np.mean(t2_forwards.overall) # t2 frwd shooting avg

    t1_defender_avg = np.mean(t1_defenders.overall) # t1 defender stats
    t2_defender_avg = np.mean(t2_defenders.overall) # t2 defender stats

    # t1 - t2

    t1_total = t1_forward_avg + t1_mid_avg + t1_gk_avg + t1_sub_avg + t1_defender_avg
    t2_total = t2_forward_avg + t2_mid_avg + t2_gk_avg + t2_sub_avg + t2_defender_avg

    difference = t1_total - t2_total
    
    return difference

In [141]:
def interval_finder(row):
    interval = ""
    difference = row["Difference"]
    if (difference > 0):
        if (difference < 5 and difference > 0):
            interval = "1 to 4"
        elif (difference < 10 and difference >= 5):
            interval = "5 to 9"
        elif (difference < 15 and difference >= 10):
            interval = "10 to 15"
        elif (difference < 20 and difference >= 15):
            interval = "15 to 20"
        elif (difference < 25 and difference >= 20):
            interval = "20 to 25"
        elif (difference < 30 and difference >= 25):
            interval = "25 to 30"
        elif (difference < 35 and difference >= 30):
            interval = "30 to 35"
        elif (difference < 40 and difference >= 35):
            interval = "35 to 40"
        elif (difference < 45 and difference >= 40):
            interval = "40 to 45"
        elif (difference < 50 and difference >= 45):
            interval = "45 to 50"
        elif (difference < 55 and difference >= 50):
            interval = "50 to 55"
        elif (difference < 60 and difference >= 55):
            interval = "55 to 60"
        elif (difference < 65 and difference >= 60):
            interval = "60 to 65"
        elif (difference > 65):
            interval = "65+"
    else:
        if (difference == 0):
            interval = "0"
        elif (difference > -5 and difference < 0):
            interval = "-1 to -5"
        elif (difference > -10 and difference <= -5):
            interval = "-5 to -10"
        elif (difference > -15 and difference <= -10):
            interval = "-10 to -15"
        elif (difference > -20 and difference <= -15):
            interval = "-15 to -20"
        elif (difference > -25 and difference <= -20):
            interval = "-20 to -25"
        elif (difference > -30 and difference <= -25):
            interval = "-25 to -30"
        elif (difference > -35 and difference <= -30):
            interval = "-30 to -35"
        elif (difference > -40 and difference <= -35):
            interval = "-35 to -40"
        elif (difference > -45 and difference <= -40):
            interval = "-40 to -45"
        elif (difference > -50 and difference <= -45):
            interval = "-45 to -50"
        elif (difference > -55 and difference <= -50):
            interval = "-50 to -55"
        elif (difference > -60 and difference <= -55):
            interval = "-55 to -60"
        elif (difference > -65 and difference <= -60):
            interval = "-60 to -65"
        elif (difference < -65):
            interval = "65-"
    return interval

In [142]:
def prob_calculator(interval):
    positive = True
    zero_checker = False
    correct_total = 0
    incorrect_total = 0
    draw_total = 0
    
    mini = matches.loc[matches.Interval == interval]
    winner_list = mini["winner"].to_list()
    total = len(mini)
    
    checker = mini["Difference"].to_list()[0]
    if checker < 0:
        positive = False
    if checker == 0:
        zero_checker = True
    
    if positive or zero_checker:
        for i in range(len(winner_list)):
            if winner_list[i] == "H":
                correct_total += 1
            elif winner_list[i] == "A":
                incorrect_total += 1
            elif winner_list[i] == "D":
                draw_total += 1
        home_odds = correct_total / total
        away_odds = incorrect_total / total
        draw_odds = draw_total / total
    if not positive:
        for i in range(len(winner_list)):
            if winner_list[i] == "A":
                correct_total += 1
            elif winner_list[i] == "H":
                incorrect_total += 1
            elif winner_list[i] == "D":
                draw_total += 1
        home_odds = incorrect_total / total
        away_odds = correct_total / total
        draw_odds = draw_total / total
    return [interval, home_odds, away_odds, draw_odds]

In [143]:
matches = pd.concat([la_liga, bundesliga, league_one, serie_a, prem])
matches

,Date,home,away,winner,Fifa
0,17/08/2018,Real Betis,Levante UD,A,fifa_20
1,17/08/2018,Girona FC,Real Valladolid CF,D,fifa_20
2,18/08/2018,FC Barcelona,Deportivo Alavés,H,fifa_20
3,18/08/2018,RC Celta,RCD Espanyol,D,fifa_20
4,18/08/2018,Villarreal CF,Real Sociedad,A,fifa_20
...,...,...,...,...,...
375,11/05/14,Norwich City,Arsenal,A,fifa_15
376,11/05/14,Southampton,Manchester United,D,fifa_15
377,11/05/14,Sunderland,Swansea City,A,fifa_15
378,11/05/14,Tottenham Hotspur,Aston Villa,H,fifa_15


In [144]:
matches["Difference"] = matches.apply(soccer_winner_predictor, axis = 1)
matches["Interval"] = matches.apply(interval_finder, axis = 1)
matches

,Date,home,away,winner,Fifa,Difference,Interval
0,17/08/2018,Real Betis,Levante UD,A,fifa_20,14.733333,10 to 15
1,17/08/2018,Girona FC,Real Valladolid CF,D,fifa_20,-11.200000,-10 to -15
2,18/08/2018,FC Barcelona,Deportivo Alavés,H,fifa_20,50.083333,50 to 55
3,18/08/2018,RC Celta,RCD Espanyol,D,fifa_20,-3.250000,-1 to -5
4,18/08/2018,Villarreal CF,Real Sociedad,A,fifa_20,5.500000,5 to 9
...,...,...,...,...,...,...,...
375,11/05/14,Norwich City,Arsenal,A,fifa_15,-33.250000,-30 to -35
376,11/05/14,Southampton,Manchester United,D,fifa_15,-27.550000,-25 to -30
377,11/05/14,Sunderland,Swansea City,A,fifa_15,-11.300000,-10 to -15
378,11/05/14,Tottenham Hotspur,Aston Villa,H,fifa_15,25.000000,25 to 30


In [147]:
interval_viz = set(matches["Interval"].to_list())

In [155]:
matches.loc[matches.Interval == ""].head(60)
interval_table = pd.DataFrame(interval_viz, columns = {"interval"})

In [156]:
matches

,Date,home,away,winner,Fifa,Difference,Interval
0,17/08/2018,Real Betis,Levante UD,A,fifa_20,14.733333,10 to 15
1,17/08/2018,Girona FC,Real Valladolid CF,D,fifa_20,-11.200000,-10 to -15
2,18/08/2018,FC Barcelona,Deportivo Alavés,H,fifa_20,50.083333,50 to 55
3,18/08/2018,RC Celta,RCD Espanyol,D,fifa_20,-3.250000,-1 to -5
4,18/08/2018,Villarreal CF,Real Sociedad,A,fifa_20,5.500000,5 to 9
...,...,...,...,...,...,...,...
375,11/05/14,Norwich City,Arsenal,A,fifa_15,-33.250000,-30 to -35
376,11/05/14,Southampton,Manchester United,D,fifa_15,-27.550000,-25 to -30
377,11/05/14,Sunderland,Swansea City,A,fifa_15,-11.300000,-10 to -15
378,11/05/14,Tottenham Hotspur,Aston Villa,H,fifa_15,25.000000,25 to 30
